In [ ]:
import os
import sys
import pystan

import numpy as np
import pandas as pd

In [ ]:
## Set up constants and file paths

utility_path = 'utilities'
if not utility_path in sys.path:
    sys.path.insert(0, utility_path)

import my_utilities
import psis
import stan_utility

In [ ]:
# data
data = pd.read_csv("../data/titanic.txt", index_col="name").drop(["row.names", 
                                                                  "home.dest", 
                                                                  "room", 
                                                                  "ticket", 
                                                                  "boat"], axis=1)
data.head()

In [ ]:
# binarize categorical variables and drop NaNs
data_binarized = pd.get_dummies(titanic_data).dropna(axis=0, how="any")
data_binarized.head()

In [ ]:
# create arrays for a stan model
y = np.array(data_binarized["survived"])
X = np.array(data_binarized[["age", 
                             "pclass_1st", 
                             "pclass_2nd", 
                             "pclass_3rd", 
                             "embarked_Cherbourg", 
                             "embarked_Queenstown", 
                             "embarked_Southampton", 
                             "sex_female"]], dtype=np.dtype(float))

In [ ]:
model = stan_utility.compile_model('logistic_regression.stan')

In [ ]:
model_data = dict(n=X.shape[0], d=X.shape[1], X=X, y=y)
fit = model.sampling(data=model_data, seed=1, control=dict(max_treedepth=20))
samples = fit.extract(permuted=True)

# Appendix

## my_utilities.py